<h1><center>Introduction to Statistical Programming in Python</center></h1>

You will be running Python code inside of a Jupyter notebook and so all code must be written in code blocks which look like this:

In [ ]:
# Code goes here

To run code in a code block, you must select the code block and press the run button at the top, or simply highlight the block and press `Ctrl+Enter`.

You can also save your progress at any point by going to __File --> Save and Checkpoint__.

As you work through this practical, you may also be presented with error codes (some are intentional, others are related to different versions, we will solve them as we encounter them in different ways). These are shown in the <span style="color: red">red boxes</span> and will help you figure out what has gone wrong. 

Here are some further useful tips and shortcuts for optimising your time with JNs: https://towardsdatascience.com/jypyter-notebook-shortcuts-bf0101a98330

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.graphics.gofplots
from statsmodels.stats.multitest import multipletests
import scipy.cluster.hierarchy as sch
from scipy import stats
from scipy.stats import ttest_rel, ttest_1samp, shapiro, chi2_contingency, linregress, kstest, anderson
from statsmodels.api import OLS, add_constant
import warnings
warnings.filterwarnings('ignore')
from packaging import version

if version.parse(np.__version__)>=version.parse('1.19.2'):
    print('numpy is up to date')
else:
    print('update numpy, current version:')
    print(np.__version__)
if version.parse(pd.__version__)>=version.parse('1.2.2'):
    print('pandas is up to date')
else:
    print('update pandas, current version:')
    print(pd.__version__)
if version.parse(scipy.__version__)>=version.parse('1.6.0'):
    print('scipy is up to date')
else:
    print('update scipy, current version:')
    print(scipy.__version__)
if version.parse(mpl.__version__)>=version.parse('3.3.4'):
    print('matplotlib is up to date')
else:
    print('update matplotlib, current version:')
    print(mpl.__version__)
if version.parse(sns.__version__)>=version.parse('0.11.1'):
    print('seaborn is up to date')
else:
    print('update seaborn, current version:')
    print(sns.__version__)
if version.parse(statsmodels.__version__)>=version.parse('0.12.1'):
    print('statsmodels is up to date')
else:
    print('update statsmodels, current version:')
    print(statsmodels.__version__)

<h3>Assignment 1</h3>

Is the overall health index normally distributed or not across local authorities in England in 2015? (dataset:  `healthIndexEngland.xlsx`,  sheet: _healthIndex2015LocalAuthority_)

Documentation: [Shapiro-Wilk](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html), [Kolmogorov-Smirnov](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html) (hint: you need another input here) and [Anderson-Darling](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html) (hint: the output is different here) tests.

In [ ]:
hi_2015 = pd.read_excel('healthIndexEngland.xlsx', sheet_name='healthIndex2015LocalAuthority')
plt.figure()
sns.histplot(hi_2015.healthIndex2015)

plt.figure()
plt.hist(hi_2015.healthIndex2015, bins=20)

If you have a different test to do, see the documentation (click links above) about the inputs and focus on one-tailed tests and change the below.

If you then still need help, use your favourite LLM to suggest code. E.g. prompt it with:

_Given the following code, and data contained in hi_2015.healthIndex2015, can you rewrite this for the Kolomogorov-Smirnov test from SciPy. Can you then explain the parts of the original code that are different?_

In [ ]:
# test for normality by using the Shapiro-Wilk test
stat, pval = shapiro(hi_2015.healthIndex2015)
print('stat=%.3f, p-value=%.3f\n' % (stat,pval))
if pval > 0.05:
    print('The data are normally distributed')
else:
    print('The data are not normally distributed')

<h3>Assignment 2</h3>

Use appropriate statistical comparison test to comment on how the health indices per domain across the regions have changed from 2015 compared with 2018, has it gotten better, worse or stayed the same? 
(dataset:  `healthIndexEngland.xlsx`,  sheet: _healthIndexPerDomainRegions_)

In [ ]:
hi_domain_region = pd.read_excel('healthIndexEngland.xlsx', sheet_name='healthIndexPerDomainRegions')
hi_domain_region_2015 = hi_domain_region.healthIndexOverall2015 
hi_domain_region_2018 = hi_domain_region['healthIndexOverall2018'] 

# test for normality by plotting the two distributions
# note that this function will be removed in a future version of seaborn
plt.figure()
#sns.distplot(hi_domain_region_2015, hist=False)
#sns.distplot(hi_domain_region_2018, bins=10, hist=True)

sns.displot(hi_domain_region_2015, kind='hist', kde=True)
sns.displot(hi_domain_region_2018, kind='hist', kde=True)

In [ ]:
# from the graph above we see that the data is normally distributed,  
# we can therefore use a parametric test to assess significance 

print(ttest_rel(hi_domain_region_2015, hi_domain_region_2018))
# using a paired t-test here because health index is a repeated measure in 2015 and 2018 
print('if you want to print just the p-value:', 
      ttest_rel(hi_domain_region_2015, hi_domain_region_2018)[1]) 

# let's compare the means now to look at differences 
if np.mean(hi_domain_region_2015) > np.mean(hi_domain_region_2018):
    print('the health index dropped in 2018')
elif np.mean(hi_domain_region_2015) == np.mean(hi_domain_region_2018):
    print('the health index has remained the same in 2015 and 2018')
else:
    print('the health index increased in 2018')
    
print(np.mean(hi_domain_region_2015))
print(np.mean(hi_domain_region_2018))

<h3>Extra Assignment A & B</h3>

Do the same as assignment 2 for local authorities, does the answer stay the same?
(dataset:  `healthIndexEngland.xlsx`,  sheet: _healthIndexPerDomainAuthority_)

Do the same as assignment 2 except now use data from all 4 years (2015 to 2018), is there a significant difference between any of the 4 years? (dataset:  `healthIndexEngland.xlsx`,  sheet: _healthIndexPerDomainRegions_)


In [ ]:
hi_domain_region = pd.read_excel('healthIndexEngland.xlsx', sheet_name='healthIndexPerDomainRegions')

# your code goes here - try to first use the above code and rewrite it.
# once you know the data format you can then prompt an LLM if easier (always try yourself first, and ask it to explain the code)

<h3>Assignment 3</h3>

Visualize deaths per week for each year, what do you observe?
(dataset:  `weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx`,  sheet: _weeklyDeaths_)


In [ ]:
deaths = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx')
deaths_object = pd.ExcelFile('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx')
print(deaths_object.sheet_names)

ee = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='East England')
em = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='East Midlands')
l = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='London')
nee = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='North East England')
nwe = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='North West England')
see = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='South East England')
swe = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='South West England')
w = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='Wales')
wm = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='West Midlands')
yth = pd.read_excel('weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx', sheet_name='Yorkshire and The Humber')


ew = ee.add(em).add(l).add(nee).add(nwe).add(see).add(swe).add(w).add(wm).add(yth)
ew.year = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
ew = ew.drop(['week'], axis=1)

plt.figure(figsize=(14,8))
for i in range(len(ew.year.values)):
    sns.lineplot(x=ew.columns[1:], y=ew.drop(['year'], axis=1).iloc[i], label=ew.iloc[i].year)
plt.legend(loc="upper right")
plt.xlabel('weeks')
plt.ylabel('deaths')
ew.iloc[:,10:25]

What do you observe for the COVID years with complete data? Any differences between these?

Do you see any other patterns over the years?

<h3>Assignment 4</h3>

Are the data normally distributed?

Apply the appropriate test to show which weeks had higher deaths in 2020 (if any), what do you observe with respect to government restrictions implemented in the UK?
(dataset: `weeklyRegisteredDeathsPerRegionPlusCovidCause.xlsx`,  sheet: _weeklyDeaths_)

In [ ]:
# test for normality first and choose between parametric and non-parametric tests

deaths_ew_2020 = ew.loc[ew['year'] == 2020].values.tolist() # this creates a nested list with the year value in as well
print(deaths_ew_2020)
deaths_ew_2020 = deaths_ew_2020[0][1:] # taking just the week values after indexing within the list
print(deaths_ew_2020) # if you print you can see the differences between the first and second list

pvals_deaths_2020 = [] # we will use this list later in multiple correction
for i in deaths_ew_2020:
    pval = ttest_1samp(deaths_ew_2020, popmean=i, alternative='less').pvalue
    pvals_deaths_2020.append(pval)

for week, pvalue in enumerate(pvals_deaths_2020):
    if pvalue <= 0.05:
        print(week+1, pvalue)


In [ ]:
# now do the same as above for 2021
deaths_ew_2021 = ew.loc[ew['year'] == 2021].values.tolist()

# your code goes here
print(deaths_ew_2021)
deaths_ew_2021 = deaths_ew_2021[0][1:] # taking just the week values after indexing within the list
print(deaths_ew_2021) # if you print you can see the differences between the first and second list

pvals_deaths_2021 = [] # we will use this list later in multiple correction
for i in deaths_ew_2021:
    pval = ttest_1samp(deaths_ew_2021, popmean=i, alternative='less').pvalue
    pvals_deaths_2021.append(pval)

for week, pvalue in enumerate(pvals_deaths_2021):
    if pvalue <= 0.05:
        print(week+1, pvalue)

<h3>Assignment 5</h3>

Visualize deaths due to respiratory causes per week for each year, what do you observe? Consider both observations across all years as well as COVID-19 specific years. 
<br>(dataset: `weeklyRegisteredDeathsEnglandWalesPlusCauses_v2021.xlsx`,  sheet: _respiratoryDiseasesAsCause_)

In [ ]:
deaths_resp_ew = pd.read_excel('weeklyRegisteredDeathsEnglandWalesPlusCauses_v2021.xlsx', sheet_name='respiratoryDiseasesAsCause')
deaths_resp_ew = deaths_resp_ew.drop(['week'], axis=1)

sns.set_context(context='notebook', font_scale=1.3)
sns.set_style('darkgrid')
sns.set_palette('Paired')
plt.figure(figsize=(14,8))
for i in range(len(ew.year.values)):
    sns.lineplot(x=deaths_resp_ew.columns[1:], y=deaths_resp_ew.drop(['year'], axis=1).iloc[i], label=ew.iloc[i].year)
plt.legend(loc="upper right")
plt.xlabel('weeks')
plt.ylabel('deaths')
deaths_resp_ew

<h3>Assignment 6</h3>

Apply appropriate test to show which weeks had higher respiratory related deaths in 2020 (if any), what do you observe?
<br>(dataset: `weeklyRegisteredDeathsEnglandWalesPlusCauses.xlsx`,  sheet: _respiratoryDiseasesAsCause_)

In [ ]:
deaths_resp_ew_2020 = deaths_resp_ew.iloc[10].values[1:].tolist()
pvals_deaths_resp_2020 = []
for i in deaths_resp_ew_2020:
    pval = ttest_1samp(deaths_resp_ew_2020, popmean=i, alternative='less').pvalue
    pvals_deaths_resp_2020.append(pval)

for week, pvalue in enumerate(pvals_deaths_resp_2020):
    if pvalue <= 0.05:
        print(week+1, pvalue)

In [ ]:
# repeat this for 2021

deaths_resp_ew_2021 = deaths_resp_ew.iloc[11].values[1:].tolist()

<h3>Extra Assignment C & D</h3>

What if you add COVID-19 deaths in 2020 to respiratory causes for England+Wales, what do you observe? 
<br>(dataset: `weeklyRegisteredDeathsEnglandWalesPlusCauses_v2021.xlsx`, sheets: _respiratoryDiseasesAsCause_ and _covidAsCause_)

What if you subtract COVID-19 deaths from total weekly deaths, what do you observe? 
<br>(dataset: `weeklyRegisteredDeathsPerRegionPlusCovidCause_v2021.xlsx`,  sheet: _weeklyDeaths_ and _covidAsCause_)

Now repeat this for 2021 as well.

In [ ]:
# your code goes here 

<h3>Assignment 7</h3>

Apply [multiple testing correction](https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html) to results from assignments 4 and 6, do the answers change? 
<br>(dataset: `weeklyRegisteredDeathsEnglandWalesPlusCauses_v2021.xlsx`,  sheets: _weeklyDeaths_ and _respiratoryDiseasesAsCause_)

Also do this for 2021.

In [ ]:
print(multipletests(pvals_deaths_2020, alpha=0.05, method='sidak'))
corr_pvals_deaths_2020 = multipletests(pvals_deaths_2020, alpha=0.05, method='hommel')[1]
for week, pvalue in enumerate(corr_pvals_deaths_2020):
    if pvalue <= 0.05:
        print(week+1, pvalue)

corr_pvals_deaths_resp_2020 = multipletests(pvals_deaths_resp_2020, alpha=0.05, method='sidak')[1]
# try with different FWER methods (see link above)

In [ ]:
# repeat these above for 2021

<h3>Extra Assignment E & F</h3>

What if you apply different multiple testing procedures, (how) do the answers change?
<br>(dataset: `weeklyRegisteredDeathsEnglandWalesPlusCauses.xlsx`,  sheets: _weeklyDeaths_ and _respiratoryDiseasesAsCause_)

Looking back at extra assignment B, if there are differences between any of the 4 years, how would you go about finding out which one(s) is/are different? 
(dataset:  `healthIndexEngland.xlsx`,  sheet: _healthIndexPerDomainRegions_)

In [ ]:
# your code goes here 

<h3>Assignment 8</h3>

Apply [$χ^2$ test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html) to find if there was a difference between regions (London vs other) in COVID-19 vs other causes as registered cause of death over all of 2020?
<br>(dataset: `weeklyRegisteredDeathsByLocationOtherCausesPlusCOVID.xlsx`,  sheet: _registeredDeathsRegions_)

In [ ]:
deaths_region = pd.read_excel('weeklyRegisteredDeathsByLocationOtherCausesPlusCOVID.xlsx', sheet_name='registeredDeathsRegions')

# taking data for the contigency table 
london_covid_deaths = deaths_region[(deaths_region.Region == 'London') & 
                                    (deaths_region.causeOfDeath == 'COVID 19')].numberOfDeaths.sum()
london_otherCause_deaths = deaths_region[(deaths_region.Region == 'London') & 
                                    (deaths_region.causeOfDeath != 'COVID 19')].numberOfDeaths.sum()
other_covid_deaths = deaths_region[(deaths_region.Region != 'London') & 
                                    (deaths_region.causeOfDeath == 'COVID 19')].numberOfDeaths.sum()
other_otherCause_deaths = deaths_region[(deaths_region.Region != 'London') & 
                                    (deaths_region.causeOfDeath != 'COVID 19')].numberOfDeaths.sum()

# quick check to see if the sums we get sum up to all deaths
print(london_covid_deaths+london_otherCause_deaths+other_covid_deaths+other_otherCause_deaths)
print(deaths_region.numberOfDeaths.sum())

# ceating a contigency table
ct = [[london_covid_deaths, london_otherCause_deaths], [other_covid_deaths, other_otherCause_deaths]]
stat, pval, dof, expected = chi2_contingency(ct)
print(stat, pval, dof, expected)

# determine association based on p-value and significance level  
if pval <= 0.05:
    print('There is a difference between London and other regions in COVID-19 vs other causes of death')
else:
    print('There seems to be no difference between London and other regions in COVID-19 vs other causes of death')


<h3>Assignment 9</h3>

Apply [$χ^2$ test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html) to find if there is a difference between care homes and hospital deaths in COVID-19 vs other causes (for England and for Wales separately)?
<br>(dataset: `weeklyRegisteredDeathsByLocationOtherCausesPlusCOVID.xlsx`,  sheet: _registeredDeathsRegions_)

In [ ]:
# print these pandas commands to look at the types of values in the locationOfRegisteredDeath and Region columns
# this will ensure we are taking the right values for the data in the contigency tables
(deaths_region.locationOfRegisteredDeath.value_counts())
(deaths_region.Region.value_counts())

# taking data for the contigency table for England
careHome_covid_deaths_e = deaths_region[(deaths_region.locationOfRegisteredDeath == 'Care home') & 
                                    (deaths_region.causeOfDeath == 'COVID 19') & 
                                        (deaths_region.Region != 'Wales')].numberOfDeaths.sum()
careHome_otherCause_deaths_e = deaths_region[(deaths_region.locationOfRegisteredDeath == 'Care home') & 
                                    (deaths_region.causeOfDeath != 'COVID 19') &
                                    (deaths_region.Region != 'Wales')].numberOfDeaths.sum()
hosp_covid_deaths_e = deaths_region[(deaths_region.locationOfRegisteredDeath == 'Hospital') & 
                                    (deaths_region.causeOfDeath == 'COVID 19') &
                                    (deaths_region.Region != 'Wales')].numberOfDeaths.sum()
hosp_otherCause_deaths_e = deaths_region[(deaths_region.locationOfRegisteredDeath == 'Hospital') & 
                                    (deaths_region.causeOfDeath != 'COVID 19') &
                                    (deaths_region.Region != 'Wales')].numberOfDeaths.sum()

# taking data for the contigency table for Wales
careHome_covid_deaths_w = deaths_region[(deaths_region.locationOfRegisteredDeath == 'Care home') & 
                                    (deaths_region.causeOfDeath == 'COVID 19') & 
                                        (deaths_region.Region == 'Wales')].numberOfDeaths.sum()
careHome_otherCause_deaths_w = deaths_region[(deaths_region.locationOfRegisteredDeath == 'Care home') & 
                                    (deaths_region.causeOfDeath != 'COVID 19') &
                                    (deaths_region.Region == 'Wales')].numberOfDeaths.sum()
hosp_covid_deaths_w = deaths_region[(deaths_region.locationOfRegisteredDeath == 'Hospital') & 
                                    (deaths_region.causeOfDeath == 'COVID 19') &
                                    (deaths_region.Region == 'Wales')].numberOfDeaths.sum()
hosp_otherCause_deaths_w = deaths_region[(deaths_region.locationOfRegisteredDeath == 'Hospital') & 
                                    (deaths_region.causeOfDeath != 'COVID 19') &
                                    (deaths_region.Region == 'Wales')].numberOfDeaths.sum()

# quick check to see if the sums we get sum up to all deaths
all_others = deaths_region[(deaths_region.locationOfRegisteredDeath != 'Hospital') & 
                                    (deaths_region.locationOfRegisteredDeath != 'Care home')].numberOfDeaths.sum()
print(careHome_covid_deaths_e + careHome_covid_deaths_w + careHome_otherCause_deaths_e + careHome_otherCause_deaths_w
     + hosp_covid_deaths_e + hosp_covid_deaths_w + hosp_otherCause_deaths_e + hosp_otherCause_deaths_w + all_others)
print(deaths_region.numberOfDeaths.sum())

# ceating a contigency table for England and Wales
ct_e = [[careHome_covid_deaths_e, careHome_otherCause_deaths_e], [hosp_covid_deaths_e, hosp_otherCause_deaths_e]]
ct_w = [[careHome_covid_deaths_w, careHome_otherCause_deaths_w], [hosp_covid_deaths_w, hosp_otherCause_deaths_w]]

stat_e, pval_e, dof_e, expected_e = chi2_contingency(ct_e)
stat_w, pval_w, dof_w, expected_w = chi2_contingency(ct_w)

# determine association based on p-value and significance level  
if pval_e <= 0.05:
    print('There is a difference between care home and hospital deaths in COVID-19 vs other causes in England')
else:
    print('There seems to be no difference between care home and hospital deaths in COVID-19 vs other causes in England')
    
if pval_w <= 0.05:
    print('There is a difference between care home and hospital deaths in COVID-19 vs other causes in Wales')
else:
    print('There seems to be no difference between care home and hospital deaths in COVID-19 vs other causes in Wales')
    

<h3>Extra Assignment G </h3>

Apply $χ^2$ tests (and adjust for multiple testing if applicable) to find if there are differences between Westminster and other local authorities in terms of COVID-19 vs other causes? Are there authorities with significantly higher/lower COVID-19-related deaths?
<br>(dataset: `weeklyRegisteredDeathsByLocationOtherCausesPlusCOVID.xlsx`,  sheet: _registeredDeathsLocalAuthority_)

In [ ]:
deaths_local = pd.read_excel('weeklyRegisteredDeathsByLocationOtherCausesPlusCOVID.xlsx', sheet_name='registeredDeathsLocalAuthority')

westminster_covid_deaths = deaths_local[(deaths_local.areaName == 'Westminster') & 
                                (deaths_local.causeOfDeath == 'COVID 19')].numberOfDeaths.sum()

# your code goes here

<h3>Assignment 10</h3>

Calculate the pairwise correlations (what [correlation](https://docs.scipy.org/doc/scipy/reference/stats.html#correlation-functions) will you use) between all items and cluster the data (what metrics), what products have similar price changes as toilet paper? And what about rice? Are there other clusters that (don’t) surprise you? What item is least similar to other items?
<br>(dataset: `highDemandProducts.xlsx`,  sheet: _highDemandProductPrices1_)

In [ ]:
products = pd.read_excel('highDemandProducts.xlsx', sheet_name='highDemandProductPrices1')

# look at how the product data are distributed
for i in products.columns[1:]:
    column_name = str(i)
    if scipy.stats.shapiro(products[column_name].dropna())[1] > 0.05:
        print('normally distributed:', column_name, shapiro(products[column_name].dropna())[1])
    else:
        print('not normally distributed:', column_name, shapiro(products[column_name].dropna())[1])

In [ ]:
products.iloc[:, 1:]

Take a look at the documentation for different options for [correlation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html), [distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html), [linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) and [cluster](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.fcluster.html) methods used below.

In [ ]:
corr_df = products.iloc[:, 1:].corr('spearman') # calculating spearman pairwise correlations using a pandas function
pdist = sch.distance.pdist(corr_df, 'correlation') # calculating pairwise distances needeed for next step
linkage = sch.linkage(pdist, method='average') # perform hierarchical/agglomerative clustering
group_ids = sch.fcluster(linkage, 0.5 * pdist.max(), 'distance') # form flat clusters from the hierarchical clustering defined by the given linkage matrix
print(group_ids)

# group the products based on clusters (for visualisation)
group1 = []
group2 = []
group3 = []
group4 = []
group5 = []
for i, c in zip(group_ids, corr_df.columns): 
    if i == 1:
        group1.append(c)
    if i == 2:
        group2.append(c)
    if i == 3:
        group3.append(c)
    if i == 4:
        group4.append(c)
    if i == 5:
        group5.append(c)
cols_ordered = group1 + group2 + group3 + group4 + group5
print(group1, group2, group3, group4, group5)

sns_df = products.iloc[:, 1:][cols_ordered].corr('spearman') # re-ordering the columns for easier visualisation in heatmap
plt.figure(figsize=(15,10))
sns.heatmap(sns_df, annot=False, cmap="RdYlBu_r", center=0) # note we changed, reversed and centered the colourmap (remove it and see what happens with defaults)


<h3>Assignment 11</h3>

Calculate a regression model to impute missing values for toilet paper using your selected variable(s), how close did your model get to the actual numbers? How about for rice? 
<br>(dataset: `highDemandProducts.xlsx`,  sheets: _highDemandProductPrices1_ and _highDemandProductPrices2_)

In [ ]:
# from above it looks like the variable correlating the most with toilet paper is tissues
products.iloc[:, 20:22]

In [ ]:
x = products['Kitchen rolls']
y = products['Toilet rolls']

mask = ~np.isnan(x) & ~np.isnan(y) # remove NaN values to make both arrays same size
slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
print("r-squared:", r_value**2)
sns.scatterplot(x, y)
sns.lineplot(x, intercept+slope*x, color='r')
plt.show()

# solve for the response variable (toilet rolls) 
y1 = intercept + slope*x[2]
y2 = intercept + slope*x[7]
y3 = intercept + slope*x[15]

# compare with actual numbers
print(y1, y2, y3)
products_nonan = pd.read_excel('highDemandProducts.xlsx', sheet_name='highDemandProductPrices2')
products_nonan.iloc[[2,7,15], 20:22]


# what if you add more than one variable for the imputation? 
# another product that correlates with toilet rolls is kithen rolls...
# hint: you will have to use a different function for multivariable regression (OLS from statsmodels.api)

<h3>Extra Assignments H & I </h3>

Instead of using the correlations and hierarchical clustering (from assignment 10) use stepwise regression (backward elimination or forward addition), how does this compare to the results from assignment 11? 

Now apply your favourite ML/DL method to do the same as in extra assignment H, how does it do? 
<br>(dataset: `highDemandProducts.xlsx`,  sheets: _highDemandProductPrices1_ and _highDemandProductPrices2_)

In [ ]:
# your code goes here

<h3>Extra Assignments J & K & L & M </h3>

Are there differences in the slopes of men and women (<1 vs 90+) between the 4 countries in the UK in 2001? What do you observe? 

Are these differences in slope persistent over the years 2002-2019?

How has the difference in life expectancy at birth between men and women changed from 2001-2019? What about for other age groups? (reminder: paired data of each local authority) 

What happens if you remove outliers and redo the example or extra assignment J? You can use your own definition, or use: exclude data from any local authority for which either men or women are in the lower or upper 0.5 percentiles. 
<br>(dataset: `lifeExpectancyUK.xlsx`,  sheets: _lifeExpectancy_)

In [ ]:
life_expectancy = pd.read_excel('lifeExpectancyUK.xlsx')
# your code goes here
sc = life_expectancy[life_expectancy.Country == 'Scotland']
en = life_expectancy[life_expectancy.Country == 'England']
wa = life_expectancy[life_expectancy.Country == 'Wales']
ni = life_expectancy[life_expectancy.Country == 'Northern Ireland']


In [ ]:
def slope_intercept(df, sex, year): # this function will return the slope and intercept (in that order) 
     return linregress(x=df[(df['Sex'] == sex) & (df['Age group'] == "'90+'")][year], 
           y=df[(df['Sex'] == sex) & (df['Age group'] == "'<1'")][year])[:2]
print('Scotland, males:', 'slope:',slope_intercept(sc, 'Male', 2001)[0], 'intercept:',slope_intercept(sc, 'Male', 2001)[1])
print(slope_intercept(sc, 'Female', 2001))
print(slope_intercept(en, 'Male', 2001))
print(slope_intercept(en, 'Female', 2001))
print(slope_intercept(wa, 'Male', 2001))
print(slope_intercept(wa, 'Female', 2001))
print(slope_intercept(ni, 'Male', 2001))
print(slope_intercept(ni, 'Female', 2001))

In [ ]:
## interaction model for Scotland

# first create the x variables
x = sc[(sc['Age group'] == "'90+'")] # the LE at 90+ for both men and wome in Scotland
x = x[['Sex', 2001]]
x['Sex'] = x['Sex'].map({'Female': 1, 'Male': 0})
x['interaction'] = x['Sex']*x[2001]

# sex column refers to the sex (1 for women, 0 for men)
# 2001 column is the LE at 90+ for both men and wome in Scotland
# interaction column is the product of first two
# print the x dataframe to look at the variables

# the y variable is the LE at <1 for both men and women
y = sc[(sc['Age group'] == "'<1'")][2001]

# resetting the indices
x = x.reset_index().drop(['index'], axis=1)
y = y.reset_index().drop(['index'], axis=1)
x = add_constant(x)
model = OLS(endog=y, exog=x).fit()

# print out the p-values for the variables
print(model.pvalues)

# can also print a complete model summary (check out the documentation) -- for some reason it does not include pvals
model.summary()

Repeat the same steps for the rest of the UK countries (England, Wales, Northern Ireland).
What do you observe?

In [ ]:
# your code goes here